
[Dataset](https://drive.google.com/file/d/1wWsrN2Ep7x6lWqOXfr4rpKGYrJhWc8z7/view)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
Root = "/content/drive/MyDrive/speech-emotion-recognition-ravdess-data"
os.chdir(Root)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
ls

In [ ]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [ ]:
def extract_feature(file_name, mfcc, chroma, mel):
    with soundfile.SoundFile(file_name) as sound_file:
        X = np.array(sound_file.read(dtype="float32"))
        sample_rate = sound_file.samplerate
        if chroma:
            stft = np.abs(librosa.stft(X))
        result = np.array([])
        if mfcc:
            mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result = np.hstack((result, mfccs))
        if chroma:
            chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
            result = np.hstack((result, chroma))
        if mel:
            mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
            result = np.hstack((result, mel))
    return result


In [ ]:
# Emotions in the RAVDESS dataset
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

#Emotions to observe
observed_emotions=['neutral','happy', 'angry', 'sad']

In [ ]:
def load_data(test_size=0.2):
    x, y = [], []
    for file in glob.glob("/content/drive/MyDrive/speech-emotion-recognition-ravdess-data/Actor_*/*.wav"):
        file_name = os.path.basename(file)
        emotion = emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        X, sample_rate = librosa.load(file)
        feature = extract_feature(file, mfcc=True, chroma=True, mel=True)
        pitch = librosa.yin(X, fmin=75, fmax=300)
        spectrogram = np.abs(librosa.stft(X))
        energy = np.mean(librosa.feature.rms(S=spectrogram), axis=0)
        combined_feature = np.concatenate((feature, [pitch.mean()], [energy.mean()]))
        x.append(combined_feature)
        y.append(emotion)

    x = np.array(x)
    return train_test_split(x, y, test_size=test_size, random_state=9)


In [ ]:
#Split the dataset
x_train,x_test,y_train,y_test=load_data(test_size=0.25)

In [ ]:
# x_train

In [ ]:
#Get the shape of the training and testing datasets
print((x_train.shape[0], x_test.shape[0]))

In [ ]:
#Get the number of features extracted
print(f'Features extracted: {x_train.shape[0]}')

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

# Define the MLPClassifier with default hyperparameters
model = MLPClassifier()

# Define the parameter grid to search over
param_grid = {
    'hidden_layer_sizes': [(100,), (200,), (300,)],
    'alpha': [0.001, 0.01, 0.1],
    'learning_rate': ['constant', 'adaptive'],
    'max_iter': [200, 500, 1000]
}

# Perform GridSearchCV
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, n_jobs=-1)
grid_search.fit(x_train, y_train)

# Use the best model for prediction
model = grid_search.best_estimator_

In [ ]:
x_train.shape

In [ ]:
np.array(y_train).shape

In [ ]:
#Train the model
model.fit(x_train,y_train)

In [ ]:
#Predict for the test set
y_pred=model.predict(x_test)

In [ ]:
y_pred

In [ ]:
#Calculate the accuracy of our model
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)

#Print the accuracy
print("Accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
f1_score(y_test, y_pred,average=None)

In [ ]:
import pandas as pd
df=pd.DataFrame({'Actual': y_test, 'Predicted':y_pred})
df.head(2)

In [ ]:
import joblib
joblib.dump(model, '/content/drive/MyDrive/model1.pkl')

In [ ]:
# Example usage
# Load the trained model

model = joblib.load('/content/drive/MyDrive/model1.pkl')

#You can put your file and test it here .

input_file = '/content/drive/MyDrive/ONE.mp3'

In [ ]:
model = joblib.load('/content/drive/MyDrive/model1.pkl')

#You can put your file and test it here .

input_file = '/content/drive/MyDrive/angry.mp3'

In [ ]:
len(input_file)

In [ ]:
def extract_features2(file_name, sr):
    # with soundfile.SoundFile(file_name) as sound_file:
  X = np.array(file_name)
  sample_rate = sr
  stft = np.abs(librosa.stft(X))
  result = np.array([])
  mfccs = np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
  result = np.hstack((result, mfccs))
  chroma = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
  result = np.hstack((result, chroma))
  mel = np.mean(librosa.feature.melspectrogram(y=X, sr=sample_rate).T, axis=0)
  result = np.hstack((result, mel))
  return result


In [ ]:
import librosa
import numpy as np

# Define chunk size (e.g., 3 seconds)
chunk_size = 3

# Load the audio file
audio, sr = librosa.load(input_file, sr=None)

# Calculate the number of samples in each chunk
chunk_samples = int(sr*chunk_size)

# Process the audio file in chunks
emotion_results = []
for i in range(0, len(audio), chunk_samples):
    chunk = audio[i:i+chunk_samples]
    # Perform feature extraction on the chunk
    feature = extract_features2(chunk, sr)  # Pass the chunk and the sample rate
    
    # Calculate pitch, spectrogram, and energy for the chunk
    pitch = librosa.yin(chunk, fmin=75, fmax=300)
    spectrogram = np.abs(librosa.stft(chunk))
    energy = np.mean(librosa.feature.rms(S=spectrogram), axis=0)
    
    # Combine the features for the chunk
    combined_feature = np.concatenate((feature, [pitch.mean()], [energy.mean()]))
    
    # Make predictions for the chunk
    x = np.array([combined_feature])
    y_pred = model.predict(x)
    
    # Store the emotion result for the chunk
    emotion_results.append(y_pred)

# Print the emotion results for each chunk
for i, emotion in enumerate(emotion_results):
    print(f"Chunk {i+1} - Predicted Emotion: {emotion}")


In [ ]:
import matplotlib.pyplot as plt

# Extract the predicted emotions from the results
emotions = [emotion[0] for emotion in emotion_results]

# Create x-axis values representing the chunk indices
x = np.arange(1, len(emotions) + 1)

# Plot the emotions
plt.plot(x, emotions)
plt.xlabel('Chunk')
plt.ylabel('Emotion')
plt.title('Emotional Changes Throughout the Audio')
plt.show()
